In [1]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np

In [2]:
class skyDataset(Dataset):
    def __init__(self, dataset_dim, data_path, label_path):
        super().__init__()
        self.data = np.memmap(data_path, dtype='float32', mode='r', shape=(dataset_dim, 3, 416, 416))
        self.label = np.memmap(label_path, dtype='uint8', mode='r', shape=(dataset_dim,))
        self.size = dataset_dim

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        image = self.data[idx]
        image = torch.from_numpy(np.array(image))

        label = torch.tensor(self.label[idx], dtype=torch.float32)

        return image, label

In [3]:
class skyModel(nn.Module):
    def __init__(self):
        super(skyModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 3, 4, 4)  # 3 out channel, 4x4 con stride 4
        self.conv2 = nn.Conv2d(3, 3, 4, 2)  # idem ma con stride 2
        self.pooling = nn.MaxPool2d(3, 2)  # 3x3 con stride 3
        self.relu = nn.ReLU()
        self.linear = nn.Linear(1875, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pooling(x)

        x = torch.flatten(x, 1)
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x


def eval_acc(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y_true in data_loader:

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)
            y_pred = torch.round(y_pred)

            total += y_true.size(0)
            c = (y_pred == y_true).sum().item()
            correct += c

    return correct / total


In [4]:
train_data = os.path.join(".","dataset", "train_data.npm")
train_label = os.path.join(".","dataset", "train_label.npm")
test_data = os.path.join(".","dataset", "test_data.npm")
test_label = os.path.join(".","dataset", "test_label.npm")

train_dataset = skyDataset(3200, train_data, train_label)
test_dataset = skyDataset(800, test_data, test_label)

dl_train = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
dl_test = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [5]:
learning_rate = 0.01
epoch_num = 10

model = skyModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_values = []
train_accs = []
test_accs = []

for i in range(epoch_num):
    for j, (x, y_true) in enumerate(dl_train):
        y_pred = model(x)
        y_pred = torch.squeeze(y_pred)

        loss = loss_fn(y_pred, y_true)
        l = loss.item()
        loss_values.append(l)
        print("Epoch:", i, "Batch:", j, "Loss:", l)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if j%50 == 0:
            train_acc = eval_acc(model, dl_train)
            test_acc = eval_acc(model, dl_test)
            print("Epoch:", i, "Batch:", j)
            print("Train acc:", train_acc)
            print("Test acc:", test_acc)
            print("\n\n\n\n\n\n")
            train_accs.append(train_acc)
            test_accs.append(test_accs)


torch.save(model.state_dict(), "skyModel.bin")
print(loss_values)
print(train_accs)
print(test_accs)

Epoch: 0 Batch: 0 Loss: 0.7007284760475159
Epoch: 0 Batch: 0
Train acc: 0.1534375
Test acc: 0.03125







Epoch: 0 Batch: 1 Loss: 0.6956275701522827
Epoch: 0 Batch: 2 Loss: 0.6935051679611206
Epoch: 0 Batch: 3 Loss: 0.6897905468940735
Epoch: 0 Batch: 4 Loss: 0.6876078844070435
Epoch: 0 Batch: 5 Loss: 0.6849812865257263
Epoch: 0 Batch: 6 Loss: 0.6819064617156982
Epoch: 0 Batch: 7 Loss: 0.6796293258666992
Epoch: 0 Batch: 8 Loss: 0.6782955527305603
Epoch: 0 Batch: 9 Loss: 0.6732543110847473
Epoch: 0 Batch: 10 Loss: 0.6692911386489868
Epoch: 0 Batch: 11 Loss: 0.6606659889221191
Epoch: 0 Batch: 12 Loss: 0.6590780019760132
Epoch: 0 Batch: 13 Loss: 0.6386867761611938
Epoch: 0 Batch: 14 Loss: 0.6154899597167969
Epoch: 0 Batch: 15 Loss: 0.5813717246055603
Epoch: 0 Batch: 16 Loss: 0.5548110604286194
Epoch: 0 Batch: 17 Loss: 0.4462732970714569
Epoch: 0 Batch: 18 Loss: 0.4377937316894531
Epoch: 0 Batch: 19 Loss: 0.3476307690143585
Epoch: 0 Batch: 20 Loss: 0.3645615577697754
Epoch: 0 Batch: 21 Los